In [1]:
import os
from dotenv import load_dotenv
from typing import Any
from pathlib import Path

#import necesssary azure modules
from azure.identity import DefaultAzureCredential
from azure.ai.agents import AgentsClient
from azure.ai.agents.models import FilePurpose, CodeInterpreterTool, ListSortOrder, MessageRole

In [2]:
#get env variables
load_dotenv()
endpoint = os.getenv('PROJECT_ENDPOINT')
model_name = os.getenv('MODEL_DEPLOYMENT_NAME')

In [3]:
#connect to the agent client
agent_client = AgentsClient(
    endpoint=endpoint,
    credential=DefaultAzureCredential(
    )
)
agent_client

In [4]:
#upload the file and code interpreter tool
file_path = 'data.txt'

file = agent_client.files.upload_and_poll(
    file_path=file_path,
    purpose=FilePurpose.AGENTS,
    filename='additional data for agent'
)

print(f'File {file.filename} has been uploaded for agent')

code_interpreter = CodeInterpreterTool(file_ids=[file.id])

File data.txt has been uploaded for agent


In [5]:
#define an agent that uses the codeinterpretertool
agent = agent_client.create_agent(
    model=model_name,
    name = 'data-agent',
    instructions='You are an AI agent that analyzes the data in the file that has been uploaded. Use Python to calculate statistical metrics as necessary.',
    tools=code_interpreter.definitions,
    tool_resources=code_interpreter.resources,
)
print(f'Using agent: {agent.name}')

Using agent: data-agent


In [6]:
#create a thread for the conversation
thread = agent_client.threads.create()

In [7]:
# Loop until the user types 'quit'
while True:
    # Get input text
    user_prompt = input("Enter a prompt (or type 'quit' to exit): ")

    if user_prompt.lower() == "quit":
        break

    if len(user_prompt) == 0:
        print("Please enter a prompt.")
        continue

    # Send a prompt to the agent
    message = agent_client.messages.create(
        thread_id=thread.id,
        role="user",
        content=user_prompt,
    )

    run = agent_client.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)


    # Check the run status for failures
    if run.status == "failed":
     print(f"Run failed: {run.last_error}")


    # Show the latest response from the agent
    last_msg = agent_client.messages.get_last_message_text_by_role(
       thread_id=thread.id,
       role=MessageRole.AGENT,
    )

    if last_msg:
       print(f"Last Message: {last_msg.text.value}")

Last Message: The category with the highest cost is "Transportation" with a cost of 2301.0.
Last Message: Here is the text-based bar chart showing costs by category:

```
Accommodation  : ############## (674.56)
Transportation : ################################################## (2301.0)
Meals          : ##### (267.89)
Misc.          :  (34.5)
```
Last Message: The standard deviation of the cost is approximately 1022.47.


In [8]:
# Get the conversation history
print("\nConversation Log:\n")

messages = agent_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)

for message in messages:
    if message.text_messages:
        last_msg = message.text_messages[-1]
        
        print(f"{message.role}: {last_msg.text.value}\n")


Conversation Log:

user: What's the category with the highest cost?

assistant: The dataset contains two columns: "Category" and "Cost". I'll determine which category has the highest cost.

assistant: The category with the highest cost is "Transportation" with a cost of 2301.0.

user: Create a text-based bar chart showing cost by category

assistant: Here is the text-based bar chart showing costs by category:

```
Accommodation  : ############## (674.56)
Transportation : ################################################## (2301.0)
Meals          : ##### (267.89)
Misc.          :  (34.5)
```

user: What's the standard deviation of cost?

assistant: The standard deviation of the cost is approximately 1022.47.



In [9]:
# Agent Clean up 
agent_client.delete_agent(agent.id)